# Notes

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from imblearn.pipeline import Pipeline 
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise

/content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise


In [ ]:
hourly_sales = pd.read_csv('data/hourly_sales_features.csv')
orig_columns = hourly_sales.columns

type conversion: convert string to datetime

In [ ]:
hourly_sales.hour = pd.to_datetime(hourly_sales.hour)


In [ ]:
hourly_sales.head()

,hour,num_of_hours,num_of_weeks,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
0,2013-11-07 13:00:00,0,0,3457.0,NaN,NaN,NaN,NaN,NaN,13,3,False,False,False,False,False,False,False,False,48.0
1,2013-11-07 14:00:00,1,0,2250.0,NaN,NaN,NaN,NaN,NaN,14,3,False,False,False,False,False,False,False,False,48.0
2,2013-11-07 15:00:00,2,0,0.0,NaN,NaN,NaN,NaN,NaN,15,3,False,False,False,False,False,False,False,False,48.0
3,2013-11-07 16:00:00,3,0,729.0,NaN,NaN,NaN,NaN,NaN,16,3,False,False,False,False,False,False,False,False,48.0
4,2013-11-07 17:00:00,4,0,0.0,NaN,NaN,NaN,NaN,NaN,17,3,False,False,False,False,False,False,False,False,48.0


In [ ]:
print(min(hourly_sales.hour))
print(max(hourly_sales.hour))

2013-11-07 13:00:00
2018-05-06 19:00:00


In [ ]:
hourly_sales.shape

(39391, 20)

In [ ]:
hourly_sales.columns

Index(['hour', 'num_of_hours', 'num_of_weeks', 'sales',
       'AR__sales_weekly_total_52weeks_ago',
       'AR__sales_weekly_total_51weeks_ago', 'AR__sales_364_DAYS_ago_same_hod',
       'AR__sales_5_weeks_ago_p_0h', 'AR__sales_6_weeks_ago_p_0h',
       'seasonal__hod', 'seasonal__dow', 'holiday__is_easter',
       'holiday__is_thanksgiving', 'holiday__is_thanksgiving_post_1d',
       'holiday__is_thanksgiving_post_2d', 'holiday__is_thanksgiving_post_3d',
       'holiday__is_christmas', 'holiday__is_christmas_post_1d',
       'holiday__between_thanksgiving_n_christmas',
       'holiday__days_to_next_christmas'],
      dtype='object')

# Feature data transformation

remove data without full features because we use AR features 1 year ago.

In [ ]:
print(hourly_sales.shape)
begin_time = hourly_sales.hour.iloc[0]
hourly_sales = hourly_sales[hourly_sales.hour > begin_time+pd.Timedelta(370,'d')]
print(hourly_sales.shape)

(39391, 20)
(30510, 20)


## keep features & labels only

In [ ]:
hourly_sales.set_index('num_of_hours',inplace = True) # useful for validation and testing
hourly_sales.drop(columns = ['num_of_weeks'], axis = 1, inplace = True) # keep column 'hour' for train test split

In [ ]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
num_of_hours,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,14,2,False,False,False,False,False,False,False,False,43.0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,15,2,False,False,False,False,False,False,False,False,43.0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,16,2,False,False,False,False,False,False,False,False,43.0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,17,2,False,False,False,False,False,False,False,False,43.0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,18,2,False,False,False,False,False,False,False,False,43.0


## Null Value check

In [ ]:
pd.set_option('display.max_rows', None)
hourly_sales.isna().sum()

hour                                         0
sales                                        0
AR__sales_weekly_total_52weeks_ago           0
AR__sales_weekly_total_51weeks_ago           0
AR__sales_364_DAYS_ago_same_hod              0
AR__sales_5_weeks_ago_p_0h                   0
AR__sales_6_weeks_ago_p_0h                   0
seasonal__hod                                0
seasonal__dow                                0
holiday__is_easter                           0
holiday__is_thanksgiving                     0
holiday__is_thanksgiving_post_1d             0
holiday__is_thanksgiving_post_2d             0
holiday__is_thanksgiving_post_3d             0
holiday__is_christmas                        0
holiday__is_christmas_post_1d                0
holiday__between_thanksgiving_n_christmas    0
holiday__days_to_next_christmas              0
dtype: int64

## Categorical & boolean Feature Transformation

In [ ]:
hourly_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30510 entries, 8881 to 39390
Data columns (total 18 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   hour                                       30510 non-null  datetime64[ns]
 1   sales                                      30510 non-null  float64       
 2   AR__sales_weekly_total_52weeks_ago         30510 non-null  float64       
 3   AR__sales_weekly_total_51weeks_ago         30510 non-null  float64       
 4   AR__sales_364_DAYS_ago_same_hod            30510 non-null  float64       
 5   AR__sales_5_weeks_ago_p_0h                 30510 non-null  float64       
 6   AR__sales_6_weeks_ago_p_0h                 30510 non-null  float64       
 7   seasonal__hod                              30510 non-null  int64         
 8   seasonal__dow                              30510 non-null  int64         
 9   holiday__is_ea

In [ ]:
# merge 1-6 oclock
hourly_sales.seasonal__hod = hourly_sales.seasonal__hod.apply(lambda x: x if x not in {1,2,3,4,5,6} else 4)
set(hourly_sales.seasonal__hod)

{0, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}

In [ ]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
num_of_hours,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,14,2,False,False,False,False,False,False,False,False,43.0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,15,2,False,False,False,False,False,False,False,False,43.0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,16,2,False,False,False,False,False,False,False,False,43.0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,17,2,False,False,False,False,False,False,False,False,43.0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,18,2,False,False,False,False,False,False,False,False,43.0


In [ ]:
# one-hot encoding for categorical features
categorical_featuers = [feature for feature in list(hourly_sales.columns) if 'seasonal' in feature]
print(categorical_featuers)
print(hourly_sales.shape)
for feature in categorical_featuers:
  hourly_sales = pd.concat(
      [hourly_sales.drop(feature,axis=1),
      pd.get_dummies(hourly_sales[feature],
                      prefix=feature+'_')],
                      axis=1)
print(hourly_sales.shape)

['seasonal__hod', 'seasonal__dow']
(30510, 18)
(30510, 42)


In [ ]:
# convert booelan to int
boolean_features = [feature for feature in list(hourly_sales.columns) if 'holiday' in feature]
if 'holiday__days_to_next_christmas' in boolean_features:
  boolean_features.remove('holiday__days_to_next_christmas')
for feature in boolean_features:
  print(feature)
print(hourly_sales.shape)
for feature in boolean_features:
  hourly_sales[feature] = hourly_sales[feature].astype(int)
print(hourly_sales.shape)

holiday__is_easter
holiday__is_thanksgiving
holiday__is_thanksgiving_post_1d
holiday__is_thanksgiving_post_2d
holiday__is_thanksgiving_post_3d
holiday__is_christmas
holiday__is_christmas_post_1d
holiday__between_thanksgiving_n_christmas
(30510, 42)
(30510, 42)


# removel days between thanksgiving and chirstmas and easter,

remove holidays and december to make linear regression better capture seaonality rather than fitting to holiday pattern.

In [ ]:
pre = hourly_sales.shape[0]

In [ ]:
# # remove easters
# remove_date = ['2013-12-31','2014-04-30','2014-08-31','2014-12-31']

# Easters_list = ['2013-03-31','2014-04-20','2015-04-05','2016-03-27','2017-04-16','2018-04-01'] + remove_date # easters from 2013 to 2018
# Easter_set = set()
# for item in Easters_list:
#   Easter_set.add(pd.to_datetime(item))

# thanksgiving_list = ['2013-11-28','2014-11-27','2015-11-26','2016-11-24','2017-11-23','2018-11-22'] 
# thanksgiving_set = set()
# for item in thanksgiving_list:
#   thanksgiving_set.add(pd.to_datetime(item))

# easter_n_thanksgiving = set.union(Easter_set,thanksgiving_set)
# mask = hourly_sales.hour.apply(lambda x: True if x.floor('d') not in easter_n_thanksgiving else False)

# # remove December 
# mask = mask & hourly_sales.hour.apply(lambda x: True if x.month !=12 else False)
# hourly_sales = hourly_sales[mask]

In [ ]:
post = hourly_sales.shape[0]
(pre-post)/24

0.0

In [ ]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__4,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


# Split dataset to train and test sets

Data is very valuable in this dataset, I split the dataset into only train, test sets, rather than train, dev, test sets.

In [ ]:
# remove hourly data not with full day from the last day
print(hourly_sales.hour.iloc[-1])
hourly_sales = hourly_sales[hourly_sales.hour <  pd.to_datetime(hourly_sales.hour.iloc[-1].date()) ]
print(hourly_sales.hour.iloc[-1])

2018-05-06 19:00:00
2018-05-05 23:00:00


In [ ]:
total_days = (hourly_sales.hour.iloc[-1] - hourly_sales.hour.iloc[0]).days
print('total_days: ', total_days)
print('30% days: ', int(total_days * 0.30))
test_days = 180
train_days = total_days - test_days
print('used train_days: ', train_days)
# print('used dev_days: ', dev_days)
print('used test_days: ', test_days)

total_days:  1270
30% days:  381
used train_days:  1268
used test_days:  2


In [ ]:
train_data = hourly_sales[hourly_sales.hour <= hourly_sales.hour.iloc[-1]-pd.Timedelta(test_days,'d') ]
test_data = hourly_sales[hourly_sales.hour > hourly_sales.hour.iloc[-1]-pd.Timedelta(test_days,'d' )]
print('train data range')
print(train_data.hour.iloc[0])
print(train_data.hour.iloc[-1])
print()
print('test data range')
print(test_data.hour.iloc[0])
print(test_data.hour.iloc[-1])

train data range
2014-11-12 14:00:00
2018-05-03 23:00:00

test data range
2018-05-04 00:00:00
2018-05-05 23:00:00


In [ ]:
train_data.head()


,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__4,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
# replicate holidays  (same effect of oversampling) and test the performance
# christmas
# christmas_rows = train_data[(train_data.hour.dt.month==12) & (train_data.hour.dt.day==25)]
# train_data = train_data.append([christmas_rows]*1000, ignore_index=False)

In [ ]:
print('test_date has {} days'.format(test_data.shape[0]/24))

test_date has 2.0 days


In [ ]:
columns_to_remove = []
for column in train_data.columns:
  if 'holiday__' in column or 'AR__' in column:
    columns_to_remove.append(column)
train_data = train_data.drop(columns = columns_to_remove, axis = 1)
test_data = test_data.drop(columns = columns_to_remove, axis = 1)

In [ ]:
train_data.columns

Index(['hour', 'sales', 'seasonal__hod__0', 'seasonal__hod__4',
       'seasonal__hod__7', 'seasonal__hod__8', 'seasonal__hod__9',
       'seasonal__hod__10', 'seasonal__hod__11', 'seasonal__hod__12',
       'seasonal__hod__13', 'seasonal__hod__14', 'seasonal__hod__15',
       'seasonal__hod__16', 'seasonal__hod__17', 'seasonal__hod__18',
       'seasonal__hod__19', 'seasonal__hod__20', 'seasonal__hod__21',
       'seasonal__hod__22', 'seasonal__hod__23', 'seasonal__dow__0',
       'seasonal__dow__1', 'seasonal__dow__2', 'seasonal__dow__3',
       'seasonal__dow__4', 'seasonal__dow__5', 'seasonal__dow__6'],
      dtype='object')

In [ ]:
train_data_x = train_data.drop(columns = ['hour','sales'], axis = 1)
train_data_y = train_data['sales']

test_data_x = test_data.drop(columns = ['hour','sales'], axis = 1)
test_data_y = test_data['sales']

In [ ]:
print(train_data_x.shape)
print(train_data_y.shape)
print(test_data_x.shape)
print(test_data_y.shape)

(30442, 26)
(30442,)
(48, 26)
(48,)


# remove seasonality

In [ ]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__4,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
scaler = StandardScaler()
scaler.fit(train_data_x)
train_data_x_scaled = scaler.transform(train_data_x)

## linear regression on training set

In [ ]:
train_data_x.head()

,seasonal__hod__0,seasonal__hod__4,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
#-----linear regression
regressor = RandomForestRegressor() #alpha=1.0  Ridge LinearRegression RandomForestRegressor
# regressor = LinearRegression()#alpha=1.0  Ridge
# training model using train_set
regressor.fit(train_data_x_scaled,train_data_y)
# performance on training set
y_predicted = regressor.predict(train_data_x_scaled)
rmse = mean_squared_error(train_data_y, y_predicted, squared=False) # RMS
print(regressor.__class__.__name__, ':training RMSE is (%.3f)'%rmse)
linear_regressor = regressor
y_predicted_train = y_predicted

RandomForestRegressor :training RMSE is (477.431)


## linear regression on test set

In [ ]:
test_data_x_scaled = scaler.transform(test_data_x)

In [ ]:
#-----linear regression
# performance on training set
y_predicted = linear_regressor.predict(test_data_x_scaled)
rmse = mean_squared_error(test_data_y, y_predicted, squared=False) # RMS
print(linear_regressor.__class__.__name__, ':test RMSE is (%.3f)'%rmse)
y_predicted_test = y_predicted

RandomForestRegressor :test RMSE is (281.006)


# calcualte & save residuals

In [ ]:
# verify if indexes are identical 
merged_index = list(train_data.index) + list(test_data.index)
orig_index = list(hourly_sales.index)
print(orig_index == merged_index)

True


In [ ]:
predicted_all = np.concatenate([y_predicted_train,y_predicted_test],axis = 0)
truth_all = np.concatenate([train_data_y,test_data_y],axis = 0)

In [ ]:
hourly_sales.head()

,hour,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas,seasonal__hod__0,seasonal__hod__4,seasonal__hod__7,seasonal__hod__8,seasonal__hod__9,seasonal__hod__10,seasonal__hod__11,seasonal__hod__12,seasonal__hod__13,seasonal__hod__14,seasonal__hod__15,seasonal__hod__16,seasonal__hod__17,seasonal__hod__18,seasonal__hod__19,seasonal__hod__20,seasonal__hod__21,seasonal__hod__22,seasonal__hod__23,seasonal__dow__0,seasonal__dow__1,seasonal__dow__2,seasonal__dow__3,seasonal__dow__4,seasonal__dow__5,seasonal__dow__6
num_of_hours,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8881,2014-11-12 14:00:00,3349.0,223513.0,215608.0,3361.0,2310.0,3008.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8882,2014-11-12 15:00:00,3218.0,223513.0,215608.0,3196.0,3000.0,3047.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8883,2014-11-12 16:00:00,3207.0,223513.0,215608.0,2543.0,2264.0,3363.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
8884,2014-11-12 17:00:00,3148.0,223513.0,215608.0,2425.0,3006.0,2370.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
8885,2014-11-12 18:00:00,3029.0,223513.0,215608.0,2259.0,2323.0,2994.0,0,0,0,0,0,0,0,0,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
#
sales = pd.concat([train_data,test_data])
sales = sales[['hour','sales']]
sales.rename(columns = {'hour':'Date','sales': 'Value'},inplace = True)
# replace original value by residuals
sales.Value =  truth_all - predicted_all  #predicted_all
sales.head()

,Date,Value
num_of_hours,,
8881,2014-11-12 14:00:00,165.993604
8882,2014-11-12 15:00:00,136.658771
8883,2014-11-12 16:00:00,167.916207
8884,2014-11-12 17:00:00,156.320320
8885,2014-11-12 18:00:00,152.458362


# differencing 364 days

In [ ]:
sales.head()

,Date,Value
num_of_hours,,
8881,2014-11-12 14:00:00,165.993604
8882,2014-11-12 15:00:00,136.658771
8883,2014-11-12 16:00:00,167.916207
8884,2014-11-12 17:00:00,156.320320
8885,2014-11-12 18:00:00,152.458362


In [ ]:
start_hours = 364*24
sales['new_value'] = np.nan
for i in range(start_hours, sales.shape[0]):
  num_of_hours = sales.index[i]
  sales.loc[num_of_hours,'new_value'] = sales.loc[num_of_hours,'Value'] - sales.loc[num_of_hours-start_hours,'Value']


In [ ]:
print(sales.shape[0])
sales = sales[sales.index >= sales.index[start_hours]]
print(sales.shape[0])

30490
21754


In [ ]:
sales.Value = sales.new_value
sales.drop(columns = ['new_value'], axis = 1, inplace = True)

# Save data for visulization

In [ ]:
sales['hour'] = sales.Date.dt.floor('h')
sales['day'] = sales.Date.dt.floor('d')
sales['month'] = sales.Date.dt.to_period('m').dt.to_timestamp()
sales['year'] = sales.Date.dt.to_period('y').dt.to_timestamp()

# week
sales['num_of_hours']= (sales.hour-sales.hour.iloc[0]).apply(lambda x: int(x/pd.Timedelta(1, "h")))
sales['num_of_days']= (sales.day-sales.day.iloc[0]).dt.days
sales['num_of_weeks'] = sales['num_of_days']//7

In [ ]:

# group by  'hour' sales traffic, # can use min_count=4 as parameter of sum to not count nan value
hourly_sales = sales.groupby(['hour'], as_index = False).agg(OrderedDict([('Value','sum'),
                                                                          ('num_of_hours','min'),
                                                                          ('num_of_weeks','min')
                                                                          ])) 
hourly_sales.sort_values(by = 'hour',inplace = True)


# group by 'day'
daily_sales = sales.groupby(['day'], as_index = False).agg(OrderedDict([('Value','sum')]))
daily_sales.sort_values(by = 'day',inplace = True)

# group by 'week'
weekly_sales = sales.groupby(['num_of_weeks'], as_index = False).agg(OrderedDict([('Value','sum')]))
weekly_sales.sort_values(by = 'num_of_weeks',inplace = True)

# group by 'month'
monthly_sales = sales.groupby(['month'], as_index = False).agg(OrderedDict([('Value','sum')]))
monthly_sales.sort_values(by = 'month',inplace = True)

# group by 'year'
yearly_sales = sales.groupby(['year'], as_index = False).agg(OrderedDict([('Value','sum')]))
yearly_sales.sort_values(by = 'year',inplace = True)


In [ ]:
hourly_sales.to_csv('data/hourly_sales.csv',index=False)
daily_sales.to_csv('data/daily_sales.csv',index=False)
weekly_sales.to_csv('data/weekly_sales.csv',index=False)
monthly_sales.to_csv('data/monthly_sales.csv',index=False)
yearly_sales.to_csv('data/yearly_sales.csv',index=False)

In [ ]:
hourly_sales.head()

,hour,Value,num_of_hours,num_of_weeks
0,2015-11-11 14:00:00,112.0,0,0
1,2015-11-11 15:00:00,396.0,1,0
2,2015-11-11 16:00:00,229.0,2,0
3,2015-11-11 17:00:00,298.0,3,0
4,2015-11-11 18:00:00,448.0,4,0


In [ ]:
train_data.columns

Index(['hour', 'sales', 'seasonal__hod__0', 'seasonal__hod__4',
       'seasonal__hod__7', 'seasonal__hod__8', 'seasonal__hod__9',
       'seasonal__hod__10', 'seasonal__hod__11', 'seasonal__hod__12',
       'seasonal__hod__13', 'seasonal__hod__14', 'seasonal__hod__15',
       'seasonal__hod__16', 'seasonal__hod__17', 'seasonal__hod__18',
       'seasonal__hod__19', 'seasonal__hod__20', 'seasonal__hod__21',
       'seasonal__hod__22', 'seasonal__hod__23', 'seasonal__dow__0',
       'seasonal__dow__1', 'seasonal__dow__2', 'seasonal__dow__3',
       'seasonal__dow__4', 'seasonal__dow__5', 'seasonal__dow__6'],
      dtype='object')